In [2]:
using ClusterManagers
addprocs_htc(1)

Submitting job(s).
Waiting for 1 workers: 1 .


1-element Array{Int64,1}:
 2

In [14]:
@everywhere println(Threads.nthreads())

4
	From worker 2:	6


In [17]:
@everywhere begin 
    import Distributions: Uniform
    using DiffEqBase, OrdinaryDiffEq, DiffEqMonteCarlo

    # number of iterations
    n_iter = 100000

    # time 
    tspan = (0., 100.)

    # time step
    step = 1.

    # diffeq system 
    pf_func = function (t,u,p,du)
        vel = p[3]*p[2]*u[2]/(p[4]+u[2])
        vabs = p[1]*p[5]*u[1]
        du[1] = -vabs/p[1]
        du[2] = (vabs-vel)/p[3]
    end
                     
    u0 = [C0=10., C1=0.]
    params = [Default=1., kcat=7.282813exp(-1), Vd=5.275000exp(0), Km=5.257432exp(0), kabs=2.090394exp(0)]

    pf = ParameterizedFunction(pf_func,params)
    prob = ODEProblem(pf,u0,tspan)

    # MonteCarlo 
    prob_func1 = function (prob, i)
        prob.f.params[2] = rand(Uniform(6.5, 7.5))
        prob.f.params[3] = rand(Uniform(4.5, 5.5))
        prob.f.params[4] = rand(Uniform(4.5, 5.5))
        prob.f.params[5] = rand(Uniform(1.5, 3.))
        prob
    end
    monte_prob = MonteCarloProblem(prob,prob_func=prob_func1)
end

    # solution
@everywhere @time sim1 = solve(monte_prob, Tsit5(), saveat=collect(tspan[1]:step:tspan[2]), parallel_type=:threads, num_monte=n_iter)
@everywhere @time m_val = timeseries_steps_mean(sim1)

 	From worker 2:	 24.766722 seconds (61.27 M allocations: 3.999 GB, 49.61% gc time)
25.438723 seconds (61.79 M allocations: 4.081 GB, 47.69% gc time)
 11.168744 seconds (20.24 M allocations: 1.206 GB, 50.27% gc time)
	From worker 2:	 13.091963 seconds (20.24 M allocations: 1.206 GB, 53.31% gc time)


In [11]:
using ParallelDataTransfer
for i in workers()
    m_val.u = map(+, m_val.u, @getfrom i m_val.u)
end

m_val.u /= nprocs()

101-element Array{Array{Float64,1},1}:
 [10.0,0.0]                 
 [1.1554,0.837089]          
 [0.157673,0.330416]        
 [0.0244109,0.0987743]      
 [0.00412058,0.0270368]     
 [0.000736826,0.00718382]   
 [0.000137029,0.00189064]   
 [2.62033e-5,0.000496711]   
 [5.11598e-6,0.000130723]   
 [1.01489e-6,3.45226e-5]    
 [2.04141e-7,9.15834e-6]    
 [4.16345e-8,2.44269e-6]    
 [8.62329e-9,6.55957e-7]    
 ⋮                          
 [-8.28918e-11,-2.83659e-10]
 [-1.51661e-9,7.00038e-10]  
 [-1.11225e-9,5.88741e-10]  
 [4.66399e-10,-5.32508e-10] 
 [8.15739e-10,2.36256e-10]  
 [1.66905e-9,-1.13744e-9]   
 [-4.66381e-10,6.71287e-10] 
 [-3.32646e-10,-2.10506e-10]
 [-8.05628e-10,3.6989e-10]  
 [-1.5611e-9,6.61249e-10]   
 [2.57184e-8,-2.465e-8]     
 [1.22047e-7,-7.9528e-8]    

In [16]:
using Plots 
plotly()
plot(m_val, xlims = (0.,5.), xlabel="t", ylabel="C0 and C1")